# Publication Topic Analysis 

## Objective
### A descriptive study of publications across OSU have been performed to understand the variety and depth of research across the university regarding Covid-19.

## Data Source
### Elements Coronavirus Publications report

## Scope of Analysis
### TimeFrame: 3/1/2020 - Present

### Importing required packages

In [60]:
import pandas as pd
import string
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from matplotlib import cm
from mycolorpy import colorlist as mcp
import squarify 

C:\Users\ammitra\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Importing the base dataset

In [61]:
df = pd.read_csv(r'C:\Users\ammitra\OneDrive - Oklahoma A and M System\Experts\Coronavirus_Output\Data\Coronavirus Publications - generated 2021-12-09 10-34.csv')

C:\Users\ammitra\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Data Cleaning Operations
#### a) Subset all publications after March 1st 2020 based on Publication date.
#### b) The title and abstract column have been cleaned to remove punctuations and convert them to lowercase. The extra white spaces have also been removed.
#### c) Author's department name has been combined with their First and Last Name.
#### d) The topic and abstract column have been combined for performing topic modelling on it.

In [62]:
df['publication-date'] = pd.to_datetime(df['publication-date'])
df = df[df['publication-date'] >= '3/1/2020']

C:\Users\ammitra\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [63]:
df['title_cleaned'] = df['title'].str.replace('[{}]'.format(string.punctuation), '').str.lower()
df['title_cleaned'] = df['title_cleaned'].str.strip()
df['Full Name'] = df['First Name'] +' '+df['Last Name']+ ' ( '+ df['Primary Group Descriptor']+' ) '

df_topic = df[['ID','Publication ID','title_cleaned','abstract']]
df_topic['abs_cleaned'] = df_topic['abstract'].str.replace('[{}]'.format(string.punctuation), '').str.lower()
df_topic['abs_cleaned'] = df_topic['abs_cleaned'].str.strip()
df_topic.abs_cleaned = df_topic.abs_cleaned.fillna('')
df_topic['topic_abstract'] = df_topic['title_cleaned']  + df_topic['abs_cleaned']

C:\Users\ammitra\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-63-9bcdb0035466>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['title_cleaned'] = df['title'].str.replace('[{}]'.format(string.punctuation), '').str.lower()
<ipython-input-63-9bcdb0035466>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df_topic['abs_cleaned'] = df_topic['abstract'].str.replace('[{}]'.format(string.punctuation), '').str.lower()
<ipython-input-63-9bcdb0035466>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = va

### Topic Modelling
#### Algorithm: Latent Dirichlet Allocation (LDA): A statistical model which helps distribute the documents into a fixed number of topics.

#### Source: https://towardsdatascience.com/light-on-math-machine-learning-intuitive-guide-to-latent-dirichlet-allocation-437c81220158

### Importing required packages

In [64]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
#import pyLDAvis
#import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

C:\Users\ammitra\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


### Removing Stopwords

In [65]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

### Preprocessing text for Modelling

In [66]:
data = df_topic.topic_abstract.values.tolist()

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

#print(data_words[:1])

### Identifying phrases through bigrams and trigrams

In [67]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=50) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=50)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)


### Defining functions for Text Cleaning

In [68]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [69]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

#print(data_lemmatized[:1])

### Creating the bag of words model to represent the text

In [70]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
#print(corpus[:1])

In [71]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('also', 1),
  ('analysis', 1),
  ('associate', 1),
  ('current', 1),
  ('detect', 1),
  ('different', 1),
  ('different_geographical', 1),
  ('essential', 1),
  ('flexibility', 1),
  ('gene', 3),
  ('gene_possibly', 1),
  ('genomes_showe', 1),
  ('identify', 1),
  ('include', 1),
  ('indicate', 1),
  ('influences_conformational', 1),
  ('location', 1),
  ('motion', 1),
  ('mutation', 3),
  ('oklahoma_unite', 2),
  ('orf', 1),
  ('pandemic', 3),
  ('pathogenicity', 1),
  ('peak', 1),
  ('presence', 1),
  ('report', 2),
  ('role', 1),
  ('sarscov_viruse', 2),
  ('sequence', 1),
  ('sequenced_previously', 1),
  ('similarity', 1),
  ('state', 3),
  ('statesgenomic_sequence', 1),
  ('study', 2),
  ('time_point', 1),
  ('understand', 1)]]

### Developing the LDA Model with the Bag of words representation

### Model Parameters:
#### corpus: Bag of words corpus
#### id2word: Mapping of word-id to a token
#### num_topics: Specified number of latent topics
#### chunksize: Number of documents to be used in each training chunk
#### update_every: Number of documents to be iterated through for each update
#### passes: Number of passes through the corpus during training
#### alpha: A-priori belief on document-topic distribution
#### per_word_topics: the model computes a list of topics, sorted in descending order of most likely topics for each word

#### Model Information: https://radimrehurek.com/gensim/models/ldamodel.html

In [72]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=6, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

### Identifying Top 10 words in each topic

In [73]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.020*"covid" + 0.009*"individual" + 0.009*"case" + 0.009*"state" + '
  '0.009*"vaccine" + 0.008*"analysis" + 0.008*"pandemic" + 0.008*"patient" + '
  '0.007*"impact" + 0.007*"symptom"'),
 (1,
  '0.016*"use" + 0.015*"covid" + 0.012*"news" + 0.009*"trust" + '
  '0.008*"pandemic" + 0.008*"prejudice" + 0.008*"result" + 0.007*"high" + '
  '0.007*"livestock" + 0.007*"wellbeing"'),
 (2,
  '0.014*"find" + 0.012*"firm" + 0.011*"study" + 0.011*"bocv" + '
  '0.011*"negative" + 0.010*"however" + 0.010*"pandemic" + 0.010*"viral" + '
  '0.009*"calf" + 0.009*"virus"'),
 (3,
  '0.024*"covid" + 0.022*"discrimination" + 0.016*"experience" + 0.012*"use" + '
  '0.011*"pandemic" + 0.009*"study" + 0.008*"teacher" + 0.007*"community" + '
  '0.007*"people" + 0.007*"future"'),
 (4,
  '0.023*"covid" + 0.023*"clinical" + 0.019*"disease" + 0.019*"sample" + '
  '0.015*"use" + 0.013*"infection" + 0.012*"study" + 0.011*"pool" + '
  '0.009*"result" + 0.007*"observe"'),
 (5,
  '0.022*"pandemic" + 0.022*"covid

### Visual Representation of the topics

In [74]:
import pyLDAvis.gensim_models
lda_display = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, sort_topics=False)
pyLDAvis.display(lda_display)


### Model Performance

#### Perplexity: Measurement of how well the model reproduces the held-out data 
#### Coherence: Degree of semantic similarity between high scoring words in a topic

In [75]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.298893798489513

Coherence Score:  0.25420780412743527


### Model Interpretation

#### 6 topics have been created from the data based on topic interpretability and coherence score.
#### Topic 1: covid , individual , case , state , vaccine , analysis , pandemic , patient , impact , symptom
#### Topic 2: use , covid , news , trust , pandemic , prejudice , result , high , livestock , wellbeing
#### Topic 3: find , firm , study , bocv , negative , however , pandemic , viral , calf , virus
#### Topic 4: covid , discrimination , experience , use , pandemic , study , teacher , community , people , future
#### Topic 5: covid , clinical , disease , sample , use , infection , study , pool , result , observe
#### Topic 6: pandemic , covid , study , vaccine , state , report , mutation , gene , current , mask